In [230]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from xgboost import XGBRegressor


---
---

# 0. UTILITIES

In [186]:
start_date = '2015-07-01'
end_date = '2016-06-30'

# Generate a range of dates
date_range = pd.date_range(start=start_date, end=end_date).astype(str).to_list()

date_frame = pd.DataFrame(date_range,columns=['Date'])

date_frame['Month'] = date_frame['Date'].apply(lambda x: int(x.split("-")[1]))
date_frame['Year'] = date_frame['Date'].apply(lambda x: int(x.split("-")[0]))


---
---

# 1. CLEANING

In [187]:
sales = pd.read_csv('data/raw/Sales.csv',sep='\t', low_memory=False)

In [188]:
sales['price'] = sales['MRP']
sales['Date'] = sales['Date'].apply(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y %H:%M").strftime("%Y-%m-%d"))

sales = sales[['Date','Analytic_Category','Units_sold','price']]

In [189]:
# aggregate sales

aggregated_sales = sales.groupby(['Date', 'Analytic_Category']).agg(
    Units_sold=('Units_sold', 'sum'),
    price=('price', 'sum')
).reset_index().rename({'price':'revenue'}, axis=1)


In [190]:
# explicitly write all dates and categories, with 0 where no sales took place

categories = sales['Analytic_Category'].unique()

all_combinations = pd.MultiIndex.from_product([date_range, categories], names=['Date', 'Analytic_Category'])
full_df = pd.DataFrame(index=all_combinations).reset_index()

aggregated_sales = full_df.merge(aggregated_sales, on=['Date','Analytic_Category'], how='left')

aggregated_sales['Units_sold'] = aggregated_sales['Units_sold'].fillna(0).astype(int)
aggregated_sales['revenue'] = aggregated_sales['revenue'].fillna(0).astype(float)


In [191]:
aggregated_sales

,Date,Analytic_Category,Units_sold,revenue
0,2015-07-01,CameraAccessory,0,0.0
1,2015-07-01,GamingHardware,0,0.0
2,2015-07-01,EntertainmentSmall,0,0.0
3,2015-07-01,GameCDDVD,0,0.0
4,2015-07-01,Camera,0,0.0
...,...,...,...,...
1825,2016-06-30,CameraAccessory,0,0.0
1826,2016-06-30,GamingHardware,0,0.0
1827,2016-06-30,EntertainmentSmall,0,0.0
1828,2016-06-30,GameCDDVD,0,0.0


In [192]:
# get total sales

total_sales = aggregated_sales.groupby('Date').agg(
            Analytic_Category=('Analytic_Category','first'),
            Units_sold=('Units_sold','sum'),
            revenue=('revenue','sum')
        ).reset_index()

In [193]:
total_sales['Analytic_Category'] = 'total'

In [194]:
sales = pd.concat([total_sales,aggregated_sales]).sort_values('Date')

---

In [195]:
media = pd.read_csv('data/raw/MediaInvestment.csv')

In [196]:
media = media.fillna(0.0)

In [197]:
n_days = date_frame[['Month','Date']].groupby("Month").count().reset_index()

In [198]:
media = media.merge(n_days,on='Month').rename({'Date':'Days'},axis=1)

In [199]:
media.iloc[:,2:-1] = media.iloc[:,2:-1].div(media['Days'],axis=0)

In [200]:
media = date_frame.merge(media,on=['Month','Year'], how='left').drop(['Month','Year','Days'],axis=1)

---

In [201]:
nps = pd.read_csv('data/raw/MonthlyNPSscore.csv')

In [202]:
nps['Year'] = nps['Date'].apply(lambda x: int(x.split('/')[2]))
nps['Month']= nps['Date'].apply(lambda x: int(x.split('/')[0]))
nps = nps.drop('Date',axis=1)

In [203]:
nps = date_frame.merge(nps,on=['Year','Month'],how='left').drop(['Year','Month'],axis=1)

---
---

# 2. Feature Engineering

In [204]:
features = media.merge(nps,on='Date').drop('Total Investment',axis=1)

---

### Adstock

In [205]:
def adstock_transform(series, decay):
    """Apply Adstock transformation to a media spend series."""
    adstocked = np.zeros(len(series))
    for i in range(1, len(series)):
        adstocked[i] = series[i] + decay * adstocked[i-1]
    return adstocked


In [206]:
features_adstock = features.copy()

for col in features.columns[2:]:
    features_adstock[col] = adstock_transform(features[col],0.8)

---

### Saturation

In [207]:
def hill_function(spend, theta, s):
    return (spend**s) / (spend**s + theta**s)


In [208]:
features_saturation = features_adstock.copy()

for col in features.columns[2:-1]:
    theta = features[col].mean() + features[col].std()
    features_saturation[col] = hill_function(features_adstock[col], theta, 1.5)

In [209]:
features = features_saturation

---
---

# 3. Regression

In [225]:
target = sales.query('Analytic_Category == "total"')[['Date','Units_sold']]

In [226]:
split_ratio = 0.8
train_size = int(len(features) * split_ratio)

X_train = features.iloc[:train_size].drop('Date',axis=1)
X_test = features.iloc[train_size:].drop('Date',axis=1)

y_train = target.iloc[:train_size].drop('Date',axis=1)
y_test  = target.iloc[train_size:].drop('Date',axis=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

### Linear

In [227]:
# Ridge Regression
ridge = RidgeCV(alphas=np.logspace(-3, 3, 50), store_cv_values=True)
ridge.fit(X_train_scaled, y_train)

# Lasso Regression (for feature selection)
lasso = LassoCV(alphas=np.logspace(-3, 3, 50), max_iter=50000)
lasso.fit(X_train_scaled, y_train,)

C:\Users\UY641FU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_ridge.py:2341: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(
C:\Users\UY641FU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\UY641FU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap:

LassoCV(alphas=array([1.00000000e-03, 1.32571137e-03, 1.75751062e-03, 2.32995181e-03,
       3.08884360e-03, 4.09491506e-03, 5.42867544e-03, 7.19685673e-03,
       9.54095476e-03, 1.26485522e-02, 1.67683294e-02, 2.22299648e-02,
       2.94705170e-02, 3.90693994e-02, 5.17947468e-02, 6.86648845e-02,
       9.10298178e-02, 1.20679264e-01, 1.59985872e-01, 2.12095089e-01,
       2.81176870e-01, 3.72759372e-0...
       8.68511374e-01, 1.15139540e+00, 1.52641797e+00, 2.02358965e+00,
       2.68269580e+00, 3.55648031e+00, 4.71486636e+00, 6.25055193e+00,
       8.28642773e+00, 1.09854114e+01, 1.45634848e+01, 1.93069773e+01,
       2.55954792e+01, 3.39322177e+01, 4.49843267e+01, 5.96362332e+01,
       7.90604321e+01, 1.04811313e+02, 1.38949549e+02, 1.84206997e+02,
       2.44205309e+02, 3.23745754e+02, 4.29193426e+02, 5.68986603e+02,
       7.54312006e+02, 1.00000000e+03]),
        max_iter=50000)

In [228]:
print(f"Ridge R²: {ridge.score(X_test_scaled, y_test)}")
print(f"Lasso R²: {lasso.score(X_test_scaled, y_test)}")


Ridge R²: -0.1740239525885694
Lasso R²: 0.08290042489493532


---

### XGBoost

In [231]:
xgb = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=5)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost R²: {r2_xgb:.4f}")


XGBoost R²: -0.7347
